In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

# -------------------------------
# 1. Load merged dataset
# -------------------------------
df = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2026-prediction-1/train_all.csv", low_memory=False)

# -------------------------------
# 2. Video parameters
# -------------------------------
fps = 5
width, height = 1200, 800
output_video = "/kaggle/working/nfl_synthetic_realistic.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# -------------------------------
# 3. Load background image
# -------------------------------
background = cv2.imread("/kaggle/input/other-y/Latar_NFL.png")
background = cv2.resize(background, (width, height))

# -------------------------------
# 4. Normalize coordinates
# -------------------------------
df['x_pixel'] = (df['x'] * (width/120)).astype(int)
df['y_pixel'] = (df['y'] * (height/53.3)).astype(int)

# -------------------------------
# 5. Loop per frame
# -------------------------------
repeat_factor = 5
for frame_id in sorted(df['frame_id'].unique()):
    frame = background.copy()

    df_frame = df[df['frame_id'] == frame_id]

    for _, row in df_frame.iterrows():
        x = row['x_pixel']
        y = row['y_pixel']
        side = row['player_side']

        color = (0,0,255) if side == 'home' else (255,0,0)
        cv2.circle(frame, (x, y), 10, color, -1)  # lingkaran lebih besar
        cv2.putText(frame, str(row['nfl_id']), (x+5, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    out.write(frame)

# -------------------------------
# 6. Release video
# -------------------------------
out.release()
print("✅ Realistic NFL overlay video created:", output_video)
